In [ ]:
!pip install python-dotenv
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv('.env')

# Use variables
api_key = os.getenv('API_KEY')
genai_key = os.getenv('GENAI_KEY')

In [ ]:
!pip install 'transformers<=4.34.0' openai torch scikit-learn

In [ ]:
!pip install qdrant-client[fastembed] fastembed PyMuPDF openai gradio google-generativeai
!pip install PyPDF2==1.28.6

  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached huggingface_hub-0.34.4-py3-none-any.whl (561 kB)
Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.34.0 requires tokenizers<0.15,>=0.14, but you have tokenizers 0.21.4 which is incompatible.
sentence-transfor

In [ ]:
from PyPDF2 import PdfReader, utils
import subprocess
from io import BytesIO

input_path = '/content/repaired-www-foxweather-com.pdf'

with open(input_path, 'rb') as input_file:
    input_buffer = BytesIO(input_file.read())

# Try reading the PDF directly
try:
    input_pdf = PdfReader(input_buffer)
    print("PDF read successfully.")
except utils.PdfReadError:
    # If direct reading fails, it might be a compression issue.
    print("Could not read PDF directly. Proceeding with original file.")
    # Reset buffer position for potential later use
    input_buffer.seek(0)

PDF read successfully.


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

import os
import fitz
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
def extract_text_from_pdf(pdf_path):
  doc = fitz.open(pdf_path)
  text = ""
  for page in doc:
    text+=page.get_text()
  return text

def extract_text_from_pdfs_in_directory(directory):
 for filename in os.listdir(directory):
     if filename.endswith(".pdf"):
         pdf_path = os.path.join(directory, filename)
         extracted_text = extract_text_from_pdf(pdf_path)
         txt_filename = os.path.splitext(filename)[0] + ".txt"
         txt_filepath = os.path.join(directory, txt_filename)
         with open(txt_filepath, "w") as txt_file:
             txt_file.write(extracted_text)

# Specify the directory containing PDF files
directory_path = "/content"

# Extract text from PDFs in the directory and save as text files
extract_text_from_pdfs_in_directory(directory_path)

import os
from nltk.tokenize import sent_tokenize

# List all .txt files in the directory
txt_files = [file for file in os.listdir(directory_path) if file.endswith('.txt')]

# List to store sentences from all files
all_sentences = []

# Read each text file, split into sentences, and store
for txt_file in txt_files:
 file_path = os.path.join(directory_path, txt_file)
 with open(file_path, "r") as file:
     text = file.read()
     sentences = sent_tokenize(text)
     all_sentences.extend(sentences)

# Print the first few sentences as an example
print(all_sentences[:10])  # Print first 10 sentences

["DEVELOPING STORY\nTropical Storm Erin to be Atlantic's first hurricane of 2025 with major strengthening\nlikely\nTropical Storm Erin formed in the central Atlantic Ocean on Monday and continues to head northwest.", 'WEATHER NEWS\nLEARN: What is a tropical storm?', 'EXCLUSIVE: Analysis from Bryan Norcross\nRAMPING UP: August is 2nd-busiest month of season\nPREPARE NOW: How to get ready for a hurricane\nWatch Live\nWeather News\nHow to Watch\nExtreme Weather\nLearn\nLocal Weather\nEarth & Space\nLifestyle\nDeals\nVideo\nHeavy rain and high tide in Charleston, South Carolina, prompt flood risk to the\narea\nWEATHER NEWS\nBoston declares heat emergency as temps soar in Northeast, New England\nWEATHER NEWS\nWatch: Video shows moment straight-line winds tear roof from Nebraska State\nPenitentiary\nEXTREME WEATHER\nVenus, Jupiter conjunction to wow this week but bad news for Perseid meteor\nshower peak\nEARTH & SPACE\n‘Unprecedented’ flooding swamps Milwaukee leading to water rescues as dea

In [ ]:
from fastembed import TextEmbedding
import numpy as np
import time

# Initialize the TextEmbedding model
embedding_model = TextEmbedding(model_name="BAAI/bge-base-en", cache_dir="./embeddings")

def embed_documents(documents):
 embeddings = []
 for document in documents:
     # Embed document using FastEmbed
     embedding = np.array(list((embedding_model.embed([document]))))

     # Append the embedding to the list of embeddings
     embeddings.append(embedding)

 return embeddings

# Define the documents
documents = all_sentences

# Perform embedding generation
embeddings = embed_documents(documents)
embeddings = [sublist[0] for sublist in embeddings]

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets. [_auth.py:94]


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.models import PointStruct

client = QdrantClient(
 url="https://ec069eb8-1679-4f53-971c-8fef6fe7d057.us-west-2-0.aws.cloud.qdrant.io",
 api_key="", # Replace with your actual API key
 https=True,
)
collection_name = 'fastembed_collection'
client.recreate_collection(
 collection_name=collection_name,
 vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

True

In [ ]:
client.upload_points(
 collection_name=collection_name,
 points=[
     PointStruct(
         id=idx,
         vector=vector.tolist(),
         payload={"text": text}
     )
     for idx, (vector, text) in enumerate(zip(embeddings, documents))
 ]
)

In [ ]:
from typing import List
from qdrant_client import QdrantClient
import google.generativeai as genai
import gradio as gr
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.models import ScrollResult

genai.configure(api_key="") # Replace with your actual API key
model = genai.GenerativeModel('gemini-2.5-pro')

# Function to generate completion from prompt
def generate_completion(prompt):
 response = model.generate_content(prompt)
 return response.text

# Function to embed Queries
def embed_query(Question):
 return np.array(list(embedding_model.embed([Question])))

# Initialize Qdrant Client
client = QdrantClient(
 url="https://ec069eb8-1679-4f53-971c-8fef6fe7d057.us-west-2-0.aws.cloud.qdrant.io",
 api_key="",  # Replace with your actual API key
 https=True,
)

def generate_response(Question):
 query_embeddings = embed_query(Question)
 collection_name = 'fastembed_collection'
 all_text = ""

 # Retrieve all hits and concatenate texts into a single prompt
 for query_embedding in query_embeddings:
  query_vector: List[np.ndarray] = list(query_embedding)

  hits = client.search(
      collection_name=collection_name,
      query_vector=query_vector,
      limit=50
    )
  for hit in hits:
      text = hit.payload["text"]
      all_text += text + "\n\n"

 # Generate completion using all texts as a single prompt
 prompt = f"You are a helpful chatbot. Use only the following pieces of context to answer the question. Don't make up any new information:\n\n{all_text}\n\nQuestion:{Question}\n\nAnswer:"
 completion = generate_completion(prompt)
 return completion

# Set up the Gradio interface
iface = gr.Interface(
 fn=generate_response,
 inputs=[gr.Textbox(label="Question")],  # Pass input as a list
 outputs=[gr.Textbox(label="Generated Response")],  # Pass output as a list
 title="RAG with Qdrant, FastEmbed and Gemini",
 description="Enter a question and get a generated response based on the retrieved text.",
)

iface.launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://81820c7335e7871da6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
